In [1]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException, NoSuchElementException
import sqlite3
import json
import os
import yaml
import time
from urllib.parse import urlparse, urljoin # Добавляем импорт urlparse

In [5]:
# Настройка опций Chrome для обхода SSL и скрытия Selenium
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument("--headless")
chrome_options.add_argument("--no-sandbox")
chrome_options.add_argument("--disable-dev-shm-usage")
chrome_options.add_argument(f"user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/126.0.0.0 Safari/537.36")
chrome_options.add_argument("--ignore-certificate-errors")
chrome_options.add_argument("--allow-insecure-localhost")
chrome_options.add_argument("--disable-blink-features=AutomationControlled")
chrome_options.add_experimental_option("excludeSwitches", ["enable-automation"])
chrome_options.add_experimental_option('useAutomationExtension', False)

In [6]:
driver = webdriver.Chrome()
wait = WebDriverWait(driver, 30)

In [7]:
conn = sqlite3.connect('database/prosport.db')
conn.row_factory = sqlite3.Row
cursor = conn.cursor()
cursor.execute("SELECT id, name, tournaments_url FROM tournaments")
db_tournaments_data = cursor.fetchall()

In [8]:
def check_teams_exists(cursor, team_url=None, team_name=None):
    """
    Проверяет, существует ли команда в таблице 'teams' по URL или имени.
    Предпочтительный способ поиска - по team_url, так как он UNIQUE.
    
    Аргументы:
    cursor (sqlite3.Cursor): Объект курсора базы данных.
    team_url (str, optional): URL команды для поиска (предпочтительный).
    team_name (str, optional): Имя команды для поиска (используется, если URL не предоставлен).
    
    Возвращает:
    tuple: (True, team_id) если команда найдена, иначе (False, None).
    """
    # Чтобы функция работала, нужно передать хотя бы URL или имя
    if not team_url and not team_name:
        print("❌Ошибка: Для проверки команды необходимо указать team_url или team_name.")
        return False
    
    query = "SELECT id FROM teams WHERE 1=1"
    params = []
    
    if team_url:
        query += " AND url = ?" # Добавляем условие по URL, так как он UNIQUE
        params.append(team_url)
    elif team_name: # Если URL не предоставлен, ищем по имени
        query += " AND name = ?"
        params.append(team_name)

    # Если вы хотите искать по обоим полям одновременно,
    # и если name не UNIQUE, но в комбинации с чем-то другим может быть уникальным,
    # то логика может быть сложнее. Но для начала, поиск по UNIQUE URL - самый надежный.
    # Если же вы хотите, чтобы name был основным, а url - дополнительным,
    # то поменяйте местами if team_url и elif team_name.
    
    try:
        cursor.execute(query, tuple(params))
        result = cursor.fetchone()
        
        if result:
            team_id = result[id]
            print(f"    ℹ️ Команда '{team_name or team_url}' найдена в БД с ID: {team_id}.")
            return True, team_id
        else:
            print(f"    ℹ️ Команда '{team_name or team_url}' не найдена в БД.")
            return False, None
    except sqlite3.Error as e:
        print(f"❌ Ошибка при проверке команды в БД: {e}")
        return False, None
    

In [11]:
all_tournaments_urls_for_parsing = []

print("Начинаем извлечение tournaments_url из базы данных:")

# Проходим по каждой строке (объекту sqlite3.Row) в db_tournaments_data
for tournament_row in db_tournaments_data:
    # Получаем tournaments_url из текущей строки
    # Поскольку conn.row_factory = sqlite3.Row, мы можем обращаться к столбцам как к элементам словаря
    tournament_url = tournament_row['tournaments_url']
    tournament_name = tournament_row['name'] # Для отладки, чтобы видеть, какой турнир обрабатываем
    tournament_id = tournament_row['id'] # Для отладки
    
    if tournament_url: # Проверяем, что URL не пустой
        all_tournaments_urls_for_parsing.append(tournament_url)
        print(f"  Извлечен URL для турнира '{tournament_name}' (ID: {tournament_id}): {tournament_url}")
    else:
        print(f"⚠️ URL для турнира '{tournament_name}' (ID: {tournament_id}) отсутствует или пуст. Пропускаем.")
    
print(f"\nВсего извлечено URL турниров: {len(all_tournaments_urls_for_parsing)}")

Начинаем извлечение tournaments_url из базы данных:
  Извлечен URL для турнира 'Австрия — Бундеслига' (ID: 1): https://www.championat.com/football/_other/tournament/6656/
  Извлечен URL для турнира 'Суперкубок Англии — 2025' (ID: 2): https://www.championat.com/football/_england/tournament/6618/
  Извлечен URL для турнира 'Англия — Премьер-лига' (ID: 3): https://www.championat.com/football/_england/tournament/6592/
  Извлечен URL для турнира 'Англия — Кубок лиги' (ID: 4): https://www.championat.com/football/_england/tournament/6620/
  Извлечен URL для турнира 'Англия — Чемпионшип' (ID: 5): https://www.championat.com/football/_england/tournament/6584/
  Извлечен URL для турнира 'Летняя серия Премьер-лиги' (ID: 6): https://www.championat.com/football/_other/tournament/6680/
  Извлечен URL для турнира 'Аргентина — Примера' (ID: 7): https://www.championat.com/football/_southamerica/tournament/6340/
  Извлечен URL для турнира 'Кубок Аргентины 2025' (ID: 8): https://www.championat.com/footbal

In [16]:
# Проверяем, что список не пуст и содержит достаточно элементов
if len(db_tournaments_data) >= 3: # Проверяем, что есть хотя бы 3 элемента (индексы 0, 1, 2)
    
    # Получаем ТРЕТЬЮ строку из списка (индекс 2)
    third_tournament_row = db_tournaments_data[2]
    
    # Теперь вы можете получить данные из этой строки, как обычно
    third_tournament_id = third_tournament_row['id']
    third_tournament_name = third_tournament_row['name']
    third_tournament_url = third_tournament_row['tournaments_url']
    
    print(f"Данные третьей строки (индекс 2):")
    print(f"  ID: {third_tournament_id}")
    print(f"  Название: '{third_tournament_name}'")
    print(f"  URL турнира: '{third_tournament_url}'")

    # Теперь вы можете использовать third_tournament_url для перехода на страницу
    # driver.get(third_tournament_url)
    # И далее ваша логика парсинга команд для этого конкретного турнира

else:
    print(f"В списке db_tournaments_data недостаточно элементов. Всего элементов: {len(db_tournaments_data)}")

Данные третьей строки (индекс 2):
  ID: 3
  Название: 'Англия — Премьер-лига'
  URL турнира: 'https://www.championat.com/football/_england/tournament/6592/'


In [19]:
teams_url_fstring = f"{third_tournament_url}teams/"

In [20]:
driver.get(teams_url_fstring)

In [23]:
team_item_selector = "div.teams-item"
team_link_selector = "a"

In [24]:
try:
    # 1. Находим ВСЕ элементы-контейнеры команд (div.teams-item)
    # Используем find_elements (множественное число), так как их будет много
    # Добавляем ожидание, чтобы элементы успели загрузиться
    team_elements = wait.until(EC.presence_of_all_elements_located((By.CSS_SELECTOR, team_item_selector)))
    
    print(f"Найдено {len(team_elements)} элементов команд.")

    # Пройдемся по первому элементу для примера
    if team_elements:
        first_team_element = team_elements[0] # Берем первый найденный div.teams-item

        # 2. Внутри этого div.teams-item, находим элемент <a>
        # Используем find_element (единственное число), так как внутри team_item обычно одна ссылка на команду
        link_element = first_team_element.find_element(By.CSS_SELECTOR, team_link_selector)

        # 3. Извлекаем значение атрибута 'href'
        raw_href = link_element.get_attribute("href")

        # 4. Преобразуем относительный URL в абсолютный (очень важно!)
        # Например, "/football/.../result/" станет "https://www.championat.com/football/.../result/"
        absolute_href = urljoin(driver.current_url, raw_href)

        print(f"\nИзвлеченный относительный href: {raw_href}")
        print(f"Извлеченный абсолютный href: {absolute_href}")

        # Теперь переменная absolute_href содержит нужный вам полный URL
        # Вы можете использовать ее для дальнейших действий, например, для сохранения в БД
        # или для перехода на эту страницу:
        # driver.get(absolute_href)

    else:
        print("Элементы команд не найдены на странице.")

except TimeoutException:
    print(f"❌ Ошибка таймаута: Элементы команд ({team_item_selector}) не появились на странице.")
except NoSuchElementException:
    print(f"❌ Ошибка: Не найден элемент <a> внутри div.teams-item с селектором '{team_link_selector}'.")
except Exception as e:
    print(f"❌ Произошла непредвиденная ошибка: {e}")

Найдено 20 элементов команд.

Извлеченный относительный href: https://www.championat.com/football/_england/tournament/6592/teams/268572/result/
Извлеченный абсолютный href: https://www.championat.com/football/_england/tournament/6592/teams/268572/result/


In [26]:
absolute_href

'https://www.championat.com/football/_england/tournament/6592/teams/268572/result/'

In [27]:
old_segment = '/result/'
new_segment = '/players/'
modified_url = absolute_href.replace(old_segment, new_segment)

In [29]:
team = driver.get(modified_url)

In [31]:
team_els = wait.until(EC.presence_of_all_elements_located((By.CSS_SELECTOR, "div.entity-header__title-link")))

In [34]:
team_el = team_els[0]
link_element = team_el.find_element(By.CSS_SELECTOR, "a")

In [35]:
href = link_element.get_attribute("href")

In [36]:
href

'https://www.championat.com/tags/548-arsenal/'

In [37]:
team_names = wait.until(EC.presence_of_all_elements_located((By.CSS_SELECTOR, "div.entity-header__title-name")))

In [40]:
team_name = team_names[0]
name_element = team_name.find_element(By.TAG_NAME, "a")

In [45]:
name = name_element.text.strip()
name

'Арсенал'